In [1]:
from flask import Flask, request, jsonify
from pyperiscope import Pilot
from collections import OrderedDict
import uuid
import threading
import os
from typing import Dict, Optional

app = Flask(__name__)

class ProcessManager:
    def __init__(self, max_size: int = 50):
        self.processes: OrderedDict[str, Dict] = OrderedDict()
        self.max_size = max_size
        self.lock = threading.Lock()
        self.current_process: Optional[Dict] = None
        
    def add_process(self, guid: str, state: Dict) -> None:
        with self.lock:
            if len(self.processes) >= self.max_size:
                self.processes.popitem(last=False)
            self.processes[guid] = state
            print(f"Added process with GUID: {guid}")  # Debugging
            print(f"Processes in manager: {self.processes}")  # Debugging
            
    def get_process(self, guid: str) -> Optional[Dict]:
        with self.lock:
            return self.processes.get(guid)
    
    def set_current_process(self, process: Optional[Dict]) -> None:
        with self.lock:
            self.current_process = process
            
    def get_current_process(self) -> Optional[Dict]:
        with self.lock:
            return self.current_process

process_manager = ProcessManager()

@app.route('/run', methods=['POST'])
def run_notebook():
    current_process = process_manager.get_current_process()
    if current_process:
        return jsonify({
            'status': 'busy',
            'guid': current_process['guid']
        }), 409

    data = request.get_json()
    command = data.get('command')
    
    if not command:
        return jsonify({'error': 'No command provided'}), 400

    process_guid = str(uuid.uuid4())
    
    try:
        # Change to the command directory
        original_dir = os.getcwd()
        os.chdir(command)
        
        pilot = Pilot("run.ipynb", step_timeout=2)
        
        process_info = {
            'status': 'running',
            'command': command,
            'guid': process_guid,
            'payload': None
        }
        
        process_manager.set_current_process(process_info)
        process_manager.add_process(process_guid, process_info)
        
        def run_notebook_task():
            try:
                result = pilot.run_workbook()
                process_info.update({
                    'status': 'finished',
                    'payload': result
                })
            except Exception as e:
                process_info.update({
                    'status': 'error',
                    'error': str(e)
                })
            finally:
                # Revert to the original directory
                os.chdir(original_dir)
                process_manager.set_current_process(None)
                process_manager.add_process(process_guid, process_info)
                print(f"Process {process_guid} completed with status: {process_info['status']}")  # Debugging
                
        threading.Thread(target=run_notebook_task).start()
        
        return jsonify({
            'status': 'running',
            'guid': process_guid
        })
        
    except Exception as e:
        error_info = {
            'status': 'error',
            'command': command,
            'guid': process_guid,
            'error': str(e)
        }
        process_manager.add_process(process_guid, error_info)
        return jsonify({'error': str(e)}), 500

@app.route('/status', methods=['POST'])
def get_status():
    # Try to get GUID from different locations
    guid = None
    
    # Check JSON body first
    if request.is_json:
        data = request.get_json()
        guid = data.get('guid')
    
    # If no GUID in body, check query parameters
    if not guid:
        guid = request.args.get('guid')
    
    # If still no GUID, try form data
    if not guid and request.form:
        guid = request.form.get('guid')

    if not guid:
        return jsonify({
            'error': 'No GUID provided. Send GUID either in request body or as query parameter',
            'request_body': str(request.get_data()),
            'query_params': dict(request.args),
            'content_type': request.content_type
        }), 400

    # Trim whitespace from the GUID
    guid = guid.strip()
    print(f"Looking up GUID: '{guid}'")  # Debugging

    process_info = process_manager.get_process(guid)
    
    if not process_info:
        print(f"GUID not found: '{guid}'")  # Debugging
        print(f"Available GUIDs: {list(process_manager.processes.keys())}")  # Debugging
        return jsonify({'error': 'GUID unknown'}), 404
        
    return jsonify(process_info)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.177:5000
Press CTRL+C to quit
192.168.2.177 - - [31/Jan/2025 13:20:53] "POST /run HTTP/1.1" 200 -
192.168.2.177 - - [31/Jan/2025 13:20:53] "POST /status HTTP/1.1" 200 -


Added process with GUID: 7c821e8a-f1aa-4726-8957-b92adeb8d489
Processes in manager: OrderedDict({'7c821e8a-f1aa-4726-8957-b92adeb8d489': {'status': 'running', 'command': 'odoo-onshape-draft', 'guid': '7c821e8a-f1aa-4726-8957-b92adeb8d489', 'payload': None}})
Staring step: 0
Looking up GUID: '7c821e8a-f1aa-4726-8957-b92adeb8d489'
found: 1
Staring step: 0
found: 1
Staring step: 1
Staring step: 2
found: 1
Staring step: 3
Staring step: 4
found: 1
Staring step: 5
found: 1
Staring step: 1
found: 1
Staring step: 0
found: 1


192.168.2.177 - - [31/Jan/2025 13:21:23] "POST /status HTTP/1.1" 200 -


Looking up GUID: '7c821e8a-f1aa-4726-8957-b92adeb8d489'
Staring step: 1
found: 1
Staring step: 2
found: 1
Staring step: 3
found: 1
Staring step: 4
found: 1
Found product: test-base-30
Staring step: 5
found: 1
Found dimensions: ['30', '30', '30']
Staring step: 6
found: 1
Staring step: 7
found: 1
Staring step: 8
found: 1


...

Looking up GUID: '47f78e0a-f862-4b28-b9f7-7f2b87d2c1cf'
Staring step: 1
found: 1
Staring step: 2
found: 1
Staring step: 3
try: 0/10
try: 1/10
try: 2/10
try: 3/10
try: 4/10
try: 5/10
try: 6/10
try: 7/10
try: 8/10
Could not locate the image (highest confidence = 0.480)
Staring step: 2
found: 1
Staring step: 0
found: 1


192.168.2.177 - - [31/Jan/2025 14:31:53] "POST /status HTTP/1.1" 200 -


Looking up GUID: '47f78e0a-f862-4b28-b9f7-7f2b87d2c1cf'
Added process with GUID: 47f78e0a-f862-4b28-b9f7-7f2b87d2c1cf
Processes in manager: OrderedDict({'7c821e8a-f1aa-4726-8957-b92adeb8d489': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': '7c821e8a-f1aa-4726-8957-b92adeb8d489', 'payload': None}, 'e3a5e4a5-8128-40df-ad0a-d1fb39aeb0de': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': 'e3a5e4a5-8128-40df-ad0a-d1fb39aeb0de', 'payload': None}, 'c33de746-88e8-4e4a-a1de-2d2542c4d076': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': 'c33de746-88e8-4e4a-a1de-2d2542c4d076', 'payload': None}, 'cb6ea6a4-50ac-44e7-9a99-a8ae8c79ae86': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': 'cb6ea6a4-50ac-44e7-9a99-a8ae8c79ae86', 'payload': None}, '0af3c0ba-577c-4971-930d-cd44d1b9d6ad': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': '0af3c0ba-577c-4971-930d-cd44d1b9d6ad', 'payload': None}, 'eec35384-7332-44b0-9204-9bd35c8da9da

192.168.2.177 - - [31/Jan/2025 14:32:23] "POST /status HTTP/1.1" 200 -


Looking up GUID: '47f78e0a-f862-4b28-b9f7-7f2b87d2c1cf'
